In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V
import random
import argparse

from load_data import *
from utils import *
from training import *
from models import *

In [2]:
test_set = load_dataset("yonc.test")
model = EncoderDecoder(34,10,256)
model.load_state_dict(torch.load("maml_yonc_256_5.weights"))
model.set_dicts("a e i o u A E I O U b c d f g h j k l m n p q r s t v w x z .".split())

In [5]:
test_set[0][0]

[['r', ''],
 ['xuOuu', '.xu.O.u.u.'],
 ['xur', '.xu.'],
 ['uuxu', '.u.u.xu.'],
 ['rtrO', '.rO.'],
 ['rOtuO', '.rO.tu.O.'],
 ['OtO', '.O.tO.'],
 ['ttxO', '.xO.'],
 ['Ott', '.O.'],
 ['xr', ''],
 ['uxtO', '.u.tO.'],
 ['uuxta', '.u.u.ta.'],
 ['Oaaxt', '.O.a.a.'],
 ['r', ''],
 ['Orxuu', '.O.xu.u.'],
 ['xx', ''],
 ['xrO', '.rO.'],
 ['raxr', '.ra.'],
 ['ratt', '.ra.'],
 ['Oaxat', '.O.a.xa.'],
 ['aaaOa', '.a.a.a.O.a.'],
 ['uux', '.u.u.'],
 ['OrrOr', '.O.rO.'],
 ['t', ''],
 ['xtrrt', ''],
 ['rtrxa', '.xa.'],
 ['uOxu', '.u.O.xu.'],
 ['Oxxa', '.O.xa.'],
 ['rxurO', '.xu.rO.'],
 ['Ou', '.O.u.'],
 ['xrtu', '.tu.'],
 ['aOOuu', '.a.O.O.u.u.'],
 ['tOrt', '.tO.'],
 ['aauua', '.a.a.u.u.a.'],
 ['aOxr', '.a.O.'],
 ['r', ''],
 ['aux', '.a.u.'],
 ['xOuO', '.xO.u.O.'],
 ['atrxr', '.a.'],
 ['OOrxx', '.O.O.'],
 ['aOr', '.a.O.'],
 ['xtttr', ''],
 ['uuOra', '.u.u.O.ra.'],
 ['OOuO', '.O.O.u.O.'],
 ['rOaa', '.rO.a.a.'],
 ['auuaa', '.a.u.u.a.a.'],
 ['ratO', '.ra.tO.'],
 ['OOuO', '.O.O.u.O.'],
 ['txtt', ''],
 ['aOrut

In [29]:
fo = open("maml_weights_js_variables.js", "w")

fo.write(variable_from_matrix(model.enc_lstm.wi_weights, "w_i"))
fo.write(variable_from_vector(model.enc_lstm.wi_bias, "b_i"))

fo.write(variable_from_matrix(model.enc_lstm.wf_weights, "w_f"))
fo.write(variable_from_vector(model.enc_lstm.wf_bias, "b_f"))

fo.write(variable_from_matrix(model.enc_lstm.wg_weights, "w_g"))
fo.write(variable_from_vector(model.enc_lstm.wg_bias, "b_g"))

fo.write(variable_from_matrix(model.enc_lstm.wo_weights, "w_o"))
fo.write(variable_from_vector(model.enc_lstm.wo_bias, "b_o"))

fo.write("\nvar embed_dict = {\n")

saw_prev = False

for key in model.char2ind:
    if saw_prev:
        fo.write(",\n")
    emb = stringify_tensor(model.embedding.weights[model.char2ind[key]])
    fo.write("    " + '"' + key + '"' + " : " + "math.matrix(" + emb + ")")
    
    saw_prev = True
    
    
fo.write("};\n")

fo.write(variable_from_matrix(model.dec_lstm.wi_weights, "wd_i"))
fo.write(variable_from_vector(model.dec_lstm.wi_bias, "bd_i"))

fo.write(variable_from_matrix(model.dec_lstm.wf_weights, "wd_f"))
fo.write(variable_from_vector(model.dec_lstm.wf_bias, "bd_f"))

fo.write(variable_from_matrix(model.dec_lstm.wg_weights, "wd_g"))
fo.write(variable_from_vector(model.dec_lstm.wg_bias, "bd_g"))

fo.write(variable_from_matrix(model.dec_lstm.wo_weights, "wd_o"))
fo.write(variable_from_vector(model.dec_lstm.wo_bias, "bd_o"))

fo.write(variable_from_matrix(model.dec_output.weights, "decoder_out"))
fo.write(variable_from_vector(model.dec_output.bias, "decoder_bias"))

781

In [33]:
out_strings, logits, encoding, preds, hiddens, ots, iphs, hidden_prev, its, fts, gts = model(["dog"])
out_strings

['.do..EOS.EOSEOS.EOSEOS.EOSEOS.EOSEOS.EOS']

In [3]:
train = True
avg_acc = average_acc(model, test_set[:10], lr_inner=1.0, 
                      batch_size=100, train=train, 
                      update_embeddings=True)

ValueError: too many values to unpack (expected 2)

In [4]:
avg_acc

NameError: name 'avg_acc' is not defined

In [5]:
dummy_input = ["de"]

In [6]:
model.enc_lstm.wi_weights.shape

torch.Size([256, 266])

In [7]:
def stringify_tensor(tnsr):
    list_tensor = tnsr.detach().numpy().tolist()
    comma_list = ", ".join([str(elt) for elt in list_tensor])
    vector = "[" + comma_list + "]"
    
    return vector

def stringify_matrix(mtrx):
    rows = [stringify_tensor(row) for row in mtrx]
    comma_list_rows = ", ".join(rows)
    
    full_list = "[" + comma_list_rows + "]"
    
    return full_list

def variable_from_matrix(mtrx, name):
    mtrx_string = stringify_matrix(mtrx)
    
    to_print = "const " + name + " = math.matrix(" + mtrx_string + ");\n"
    
    return to_print
    
def variable_from_vector(vector, name):
    vector_string = stringify_tensor(vector)
    
    to_print = "const " + name + " = math.matrix(" + vector_string + ");\n"
    
    return to_print
    

In [11]:
5 + 7
fo = open("maml_weights_js_variables.js", "w")

fo.write(variable_from_matrix(model.enc_lstm.wi_weights, "w_i"))
fo.write(variable_from_vector(model.enc_lstm.wi_bias, "b_i"))

fo.write(variable_from_matrix(model.enc_lstm.wf_weights, "w_f"))
fo.write(variable_from_vector(model.enc_lstm.wf_bias, "b_f"))

fo.write(variable_from_matrix(model.enc_lstm.wg_weights, "w_g"))
fo.write(variable_from_vector(model.enc_lstm.wg_bias, "b_g"))

fo.write(variable_from_matrix(model.enc_lstm.wo_weights, "w_o"))
fo.write(variable_from_vector(model.enc_lstm.wo_bias, "b_o"))


5540

In [9]:
model.set_dicts("a e i o u A E I O U b c d f g h j k l m n p q r s t v w x z .".split())

In [10]:
model.embedding.weights.shape

torch.Size([34, 10])

In [11]:
fo.write("\nvar embed_dict = {\n")

saw_prev = False

for key in model.char2ind:
    if saw_prev:
        fo.write(",\n")
    emb = stringify_tensor(model.embedding.weights[model.char2ind[key]])
    fo.write("    " + '"' + key + '"' + " : " + "math.matrix(" + emb + ")")
    
    saw_prev = True
    
    
fo.write("};\n")

3

In [12]:
model.char2ind

{'NULL': 0,
 'SOS': 1,
 'EOS': 2,
 'a': 3,
 'e': 4,
 'i': 5,
 'o': 6,
 'u': 7,
 'A': 8,
 'E': 9,
 'I': 10,
 'O': 11,
 'U': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'p': 24,
 'q': 25,
 'r': 26,
 's': 27,
 't': 28,
 'v': 29,
 'w': 30,
 'x': 31,
 'z': 32,
 '.': 33}

In [13]:
model(["dog"])

(['.do..EOS.EOSEOS.EOSEOS.EOSEOS.EOSEOS.EOS'],
 [tensor([[[-2.6509e+01, -2.6539e+01, -9.3179e+00, -1.7448e+01, -1.7831e+01,
            -1.6321e+01, -1.3484e+01, -1.8333e+01, -1.8534e+01, -1.8526e+01,
            -1.7064e+01, -1.7784e+01, -1.7287e+01, -1.5158e+01, -1.5802e+01,
            -1.0419e+01, -1.7619e+01, -1.6462e+01, -1.6535e+01, -1.4481e+01,
            -1.6832e+01, -2.0672e+01, -1.4920e+01, -1.7950e+01, -1.3512e+01,
            -1.9739e+01, -1.7165e+01, -1.6575e+01, -1.5843e+01, -1.5160e+01,
            -1.6175e+01, -1.6123e+01, -1.3928e+01, -1.2576e-04]]],
         grad_fn=<LogSoftmaxBackward>),
  tensor([[[-3.0738e+01, -3.0771e+01, -1.5566e+01, -2.1429e+01, -2.3736e+01,
            -1.9867e+01, -1.3991e+01, -2.7267e+01, -2.4977e+01, -2.2509e+01,
            -2.4382e+01, -2.1573e+01, -2.4760e+01, -1.4479e+01, -1.2144e+01,
            -3.2384e-04, -1.4234e+01, -1.5845e+01, -1.5103e+01, -1.0773e+01,
            -1.2781e+01, -1.9234e+01, -1.0626e+01, -1.6172e+01, -9.6826e+00,

In [12]:
fo.write(variable_from_matrix(model.dec_lstm.wi_weights, "wd_i"))
fo.write(variable_from_vector(model.dec_lstm.wi_bias, "bd_i"))

fo.write(variable_from_matrix(model.dec_lstm.wf_weights, "wd_f"))
fo.write(variable_from_vector(model.dec_lstm.wf_bias, "bd_f"))

fo.write(variable_from_matrix(model.dec_lstm.wg_weights, "wd_g"))
fo.write(variable_from_vector(model.dec_lstm.wg_bias, "bd_g"))

fo.write(variable_from_matrix(model.dec_lstm.wo_weights, "wd_o"))
fo.write(variable_from_vector(model.dec_lstm.wo_bias, "bd_o"))

5464

In [13]:
fo.write(variable_from_matrix(model.dec_output.weights, "decoder_out"))
fo.write(variable_from_vector(model.dec_output.bias, "decoder_bias"))

781

In [16]:
model.char2ind

{'NULL': 0,
 'SOS': 1,
 'EOS': 2,
 'a': 3,
 'e': 4,
 'i': 5,
 'o': 6,
 'u': 7,
 'A': 8,
 'E': 9,
 'I': 10,
 'O': 11,
 'U': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'p': 24,
 'q': 25,
 'r': 26,
 's': 27,
 't': 28,
 'v': 29,
 'w': 30,
 'x': 31,
 'z': 32,
 '.': 33}

In [17]:
model(["do"])[1][0]

tensor([[[-3.0128e+01, -3.0085e+01, -9.0308e+00, -1.5620e+01, -1.7085e+01,
          -1.4166e+01, -1.2249e+01, -1.8494e+01, -1.8177e+01, -1.7592e+01,
          -1.6007e+01, -1.6758e+01, -1.5217e+01, -1.8921e+01, -1.6830e+01,
          -1.1985e+01, -1.8906e+01, -2.0146e+01, -1.9777e+01, -1.8053e+01,
          -1.7694e+01, -2.0871e+01, -1.5915e+01, -2.0352e+01, -1.7058e+01,
          -2.1277e+01, -1.8803e+01, -1.7422e+01, -1.8139e+01, -1.7179e+01,
          -1.6860e+01, -1.8955e+01, -1.6675e+01, -1.3243e-04]]],
       grad_fn=<LogSoftmaxBackward>)

In [18]:
nn.NLLLoss()(model(["do"])[1][0][0],torch.LongTensor([33]))

tensor(0.0001, grad_fn=<NllLossBackward>)

In [19]:
your_var = nn.NLLLoss()(model(["do"])[1][0][0],torch.LongTensor([17]))

In [20]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


# make_dot was moved to https://github.com/szagoruyko/pytorchviz
from torchviz import make_dot

In [21]:
make_dot(your_var).view()

'Digraph.gv.pdf'

In [22]:
model.set_dicts("a e i o u A E I O U b c d f g h j k l m n p q r s t v w x z .".split())

In [23]:
loss = nn.NLLLoss()(model(["do"])[1][0][0],torch.LongTensor([33]))

In [24]:
loss.backward()

tensor([[[ 1.4190e-05, -2.8911e-08, -1.5454e-05,  2.4784e-06,  7.5777e-06,
           1.5295e-05, -1.3731e-05, -1.0707e-05,  1.0926e-05, -1.2867e-05,
           1.6812e-06, -1.6902e-05, -6.0271e-06,  2.2116e-05, -8.8155e-06,
           1.1429e-05, -4.7394e-05,  4.9429e-06, -4.0503e-05, -2.6471e-05,
          -5.0203e-05,  6.2951e-05,  3.9846e-05,  9.1023e-06,  2.1406e-05,
          -9.0905e-05, -2.7329e-06, -1.9312e-04, -2.3156e-05, -1.0191e-05,
           7.3904e-05,  2.6015e-05, -5.1333e-06,  2.1694e-05,  2.0250e-05,
          -6.7665e-05, -3.9834e-05,  4.3952e-05, -3.2675e-05,  8.5106e-06,
           2.7513e-05, -1.7731e-05,  1.4941e-05, -1.2797e-05, -4.7783e-05,
          -5.0314e-05, -9.7543e-06, -1.6481e-05,  1.1324e-05, -1.5336e-05,
           6.9944e-06, -3.3993e-06, -2.8407e-06,  9.0538e-06, -1.9335e-05,
          -1.1720e-04,  5.6702e-05,  2.7581e-05, -1.2636e-05,  9.4812e-06,
          -2.1508e-05,  9.9187e-05, -4.8948e-05,  1.3274e-05,  2.0029e-05,
           1.1203e-04, -4

In [25]:
model.dec_output.weights.grad

tensor([[-3.7447e-14, -4.3426e-14, -3.4384e-14,  ...,  3.1530e-15,
         -5.3566e-14, -9.9086e-15],
        [-3.9093e-14, -4.5335e-14, -3.5896e-14,  ...,  3.2916e-15,
         -5.5921e-14, -1.0344e-14],
        [-5.4421e-05, -6.3109e-05, -4.9969e-05,  ...,  4.5822e-06,
         -7.7846e-05, -1.4400e-05],
        ...,
        [-2.6662e-09, -3.0918e-09, -2.4481e-09,  ...,  2.2449e-10,
         -3.8138e-09, -7.0548e-10],
        [-2.6066e-08, -3.0228e-08, -2.3934e-08,  ...,  2.1948e-09,
         -3.7287e-08, -6.8972e-09],
        [ 6.0232e-05,  6.9849e-05,  5.5305e-05,  ..., -5.0715e-06,
          8.6159e-05,  1.5938e-05]])

In [26]:
loss

tensor(0.0001, grad_fn=<NllLossBackward>)

In [ ]:
presoft = model(["do"])[3][0][0].detach()
presoft

In [ ]:
exped = torch.exp(presoft)
summed = torch.sum(exped)
summed

In [ ]:

grad_bias = (exped/summed - 1)
onehot = torch.zeros_like(grad_bias)
onehot[0][33] = 1.0

grad_bias = grad_bias * onehot
grad_bias

In [ ]:
model.dec_output.bias.grad

In [ ]:
sm = (exped/summed)
mat_grad = torch.transpose(sm,0,1).expand(34,34) - torch.eye(34)

torch.matmul(mat_grad, torch.transpose(onehot,0,1))


# FIXED!!!
grad_bias = torch.matmul(onehot, torch.transpose(mat_grad,0,1))

In [ ]:
nn.LogSoftmax(dim=1)(presoft)

In [ ]:
grad_bias.shape

In [ ]:
wo = model.dec_output.weights
hidden = model(["do"])[4][0][0][0].detach()
hidden.shape

In [ ]:
wo.shape, grad_bias.shape

In [ ]:
torch.transpose(torch.mm(torch.transpose(hidden,0,1),grad_bias),0,1)

In [ ]:
model.dec_output.weights.grad


In [ ]:
new_relu_feats = torch.Tensor([[1,0,3]])
new_relu_feats.requires_grad = True
logits = torch.nn.LogSoftmax(dim=1)(new_relu_feats)
logits.retain_grad()
label = torch.LongTensor([1])
loss = torch.nn.NLLLoss()(logits, label)
loss.backward()

sm = torch.nn.Softmax(dim=1)(new_relu_feats)
dloss_dlogits = torch.Tensor([[0,-1,0]])
logits.grad # This matches above
dlogits_dnew_relu_feat = torch.Tensor([[[1-sm[0][0], -sm[0][0], -sm[0][0]], [-sm[0][1], 1-sm[0][1], -sm[0][1]], [-sm[0][2], -sm[0][2], 1-sm[0][2]]]])


torch.bmm(dlogits_dnew_relu_feat, dloss_dlogits.unsqueeze(-1)).squeeze(-1)



In [ ]:
sm

In [ ]:
dlogits_dnew_relu_feat

In [ ]:
dlogits_dnew_relu_feat

In [ ]:
# This does not entirely match above, but the middle column does (corresponding to the correct class)
new_relu_feats.grad

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V
import random
import argparse

from load_data import *
from utils import *
from training import *
from models import *

In [28]:
test_set = load_dataset("yonc.test")
model = EncoderDecoder(34,10,256)
model.load_state_dict(torch.load("maml_yonc_256_5.weights"))

In [29]:
model.set_dicts("a e i o u A E I O U b c d f g h j k l m n p q r s t v w x z .".split())
loss = nn.NLLLoss()(model(["do"])[1][0][0],torch.LongTensor([33]))
loss.backward(create_graph=True, retain_graph=True)

tensor([[[ 1.4190e-05, -2.8911e-08, -1.5454e-05,  2.4784e-06,  7.5777e-06,
           1.5295e-05, -1.3731e-05, -1.0707e-05,  1.0926e-05, -1.2867e-05,
           1.6812e-06, -1.6902e-05, -6.0271e-06,  2.2116e-05, -8.8155e-06,
           1.1429e-05, -4.7394e-05,  4.9429e-06, -4.0503e-05, -2.6471e-05,
          -5.0203e-05,  6.2951e-05,  3.9846e-05,  9.1023e-06,  2.1406e-05,
          -9.0905e-05, -2.7329e-06, -1.9312e-04, -2.3156e-05, -1.0191e-05,
           7.3904e-05,  2.6015e-05, -5.1333e-06,  2.1694e-05,  2.0250e-05,
          -6.7665e-05, -3.9834e-05,  4.3952e-05, -3.2675e-05,  8.5106e-06,
           2.7513e-05, -1.7731e-05,  1.4941e-05, -1.2797e-05, -4.7783e-05,
          -5.0314e-05, -9.7543e-06, -1.6481e-05,  1.1324e-05, -1.5336e-05,
           6.9944e-06, -3.3993e-06, -2.8407e-06,  9.0538e-06, -1.9335e-05,
          -1.1720e-04,  5.6702e-05,  2.7581e-05, -1.2636e-05,  9.4812e-06,
          -2.1508e-05,  9.9187e-05, -4.8948e-05,  1.3274e-05,  2.0029e-05,
           1.1203e-04, -4

In [30]:
onehot = torch.zeros(1,34)
onehot[0][33] = 1.0

presoft = model(["do"])[3][0][0].detach() # this is the pred vector; i.e., pre-softmax

exped = torch.exp(presoft)
summed = torch.sum(exped)

sm = (exped/summed)
mat_grad = torch.transpose(sm,0,1).expand(34,34) - torch.eye(34)

grad_bias = torch.matmul(onehot, torch.transpose(mat_grad,0,1))

grad_bias # The gradient for the decoder output layer's bias term

tensor([[ 8.2340e-14,  8.5960e-14,  1.1966e-04,  1.6452e-07,  3.8022e-08,
          7.0424e-07,  4.7883e-06,  9.2937e-09,  1.2756e-08,  2.2909e-08,
          1.1173e-07,  5.2740e-08,  2.4615e-07,  6.0619e-09,  4.9053e-08,
          6.2377e-06,  6.1550e-09,  1.7816e-09,  2.5762e-09,  1.4442e-08,
          2.0673e-08,  8.6259e-10,  1.2254e-07,  1.4489e-09,  3.9060e-08,
          5.7459e-10,  6.8244e-09,  2.7145e-08,  1.3257e-08,  3.4601e-08,
          4.7638e-08,  5.8625e-09,  5.7316e-08, -1.3250e-04]])

In [31]:
hidden = model(["do"])[4][0][0][0].detach() # this is the hidden state passed to the output layer
grad_output = torch.transpose(torch.mm(torch.transpose(hidden,0,1),grad_bias),0,1)

grad_output # The gradient for the decoder output layer's weight matrix

tensor([[-3.7447e-14, -4.3426e-14, -3.4384e-14,  ...,  3.1530e-15,
         -5.3566e-14, -9.9086e-15],
        [-3.9093e-14, -4.5335e-14, -3.5896e-14,  ...,  3.2916e-15,
         -5.5921e-14, -1.0344e-14],
        [-5.4421e-05, -6.3109e-05, -4.9969e-05,  ...,  4.5822e-06,
         -7.7846e-05, -1.4400e-05],
        ...,
        [-2.6662e-09, -3.0918e-09, -2.4481e-09,  ...,  2.2449e-10,
         -3.8138e-09, -7.0547e-10],
        [-2.6066e-08, -3.0228e-08, -2.3934e-08,  ...,  2.1948e-09,
         -3.7287e-08, -6.8972e-09],
        [ 6.0259e-05,  6.9880e-05,  5.5330e-05,  ..., -5.0738e-06,
          8.6198e-05,  1.5945e-05]])

In [32]:
wo = model.dec_output.weights

dh = torch.mm(grad_bias, wo)

tanh_c = torch.tanh(model(["do"])[4][0][1][0].detach()) # this is the hidden state passed to the output layer
dho = tanh_c * dh
ot = model(["do"])[5][0][0] # ot is fred, not ot. called ho in eq page
dho = nn.Sigmoid()(ot) * (1 - nn.Sigmoid()(ot)) * dho
#dho
model.dec_lstm.wo_bias.grad

tensor([ 1.1252e-06,  2.7780e-07,  3.6881e-07, -4.4324e-11, -8.2894e-07,
        -1.8127e-06,  6.6859e-07, -1.8536e-06, -1.1590e-07, -2.5564e-07,
        -9.7166e-07, -5.3950e-06,  5.4395e-08, -1.7995e-07, -1.5577e-07,
        -4.9091e-07, -2.9337e-08, -2.5344e-07,  2.6952e-07,  7.2421e-07,
         3.2771e-06,  1.4956e-07, -9.8751e-08,  2.2556e-06,  2.9398e-07,
        -2.4772e-07,  2.5407e-07, -7.3218e-07, -2.5331e-06,  4.1391e-08,
        -2.7878e-07,  1.0279e-07,  1.1684e-06,  6.9785e-10, -6.8815e-08,
        -3.9892e-08,  8.2673e-08, -2.6560e-07, -1.4518e-06, -4.1038e-07,
        -2.6378e-07, -4.8148e-08, -5.1663e-08,  1.2930e-07, -6.6175e-07,
         1.0288e-06,  2.0185e-07,  2.5950e-07,  2.0651e-06,  1.2648e-06,
        -1.7902e-06, -3.2374e-07,  1.0023e-06, -3.5945e-07, -5.7047e-06,
        -3.1081e-06, -3.8710e-07,  7.5741e-07,  7.9405e-08,  3.6417e-07,
         8.2215e-07, -5.2911e-08, -5.3414e-07, -2.3190e-09,  2.1969e-07,
        -5.0632e-08, -5.1575e-06,  8.5421e-07, -1.1

In [33]:
iph = model(["do"])[6][0][0]
iph.shape
dwo = torch.transpose(torch.mm(torch.transpose(iph,0,1),dho),0,1)
#model.dec_lstm.wo_weights.grad
dwo

tensor([[ 9.7592e-08, -9.1436e-08,  1.7221e-07,  ...,  6.8773e-08,
         -4.0781e-07, -1.8506e-07],
        [ 2.4076e-08, -2.2557e-08,  4.2485e-08,  ...,  1.6966e-08,
         -1.0061e-07, -4.5654e-08],
        [ 3.1978e-08, -2.9961e-08,  5.6429e-08,  ...,  2.2535e-08,
         -1.3363e-07, -6.0638e-08],
        ...,
        [ 3.5013e-09, -3.2805e-09,  6.1785e-09,  ...,  2.4674e-09,
         -1.4631e-08, -6.6393e-09],
        [ 3.2358e-08, -3.0317e-08,  5.7100e-08,  ...,  2.2803e-08,
         -1.3522e-07, -6.1359e-08],
        [-8.3430e-08,  7.8168e-08, -1.4722e-07,  ..., -5.8793e-08,
          3.4863e-07,  1.5820e-07]], grad_fn=<TransposeBackward0>)

In [34]:
#dc = ho * dh * dtanh(c)
dc = torch.sigmoid(ot) * dh * (1 - torch.pow(torch.tanh(model(["do"])[4][0][1][0].detach()),2))

In [35]:
# COVERED TO HERE

In [36]:
c_old = model(["do"])[7][1][0].detach()

dhf = c_old * dc

hf = model(["do"])[9][0][0]
dhf = nn.Sigmoid()(hf) * (1 - nn.Sigmoid()(hf)) * dhf

#model.dec_lstm.wf_bias.grad 
#dhf

In [37]:
dwf = torch.transpose(torch.mm(torch.transpose(iph,0,1),dhf),0,1)

#model.dec_lstm.wf_weights.grad 
#dwf

In [38]:
hi = model(["do"])[8][0][0]
hg = model(["do"])[10][0][0]

In [39]:
dhc = torch.sigmoid(hi) * dc
dhc = (1 - torch.pow(torch.tanh(hg),2)) * dhc

#model.dec_lstm.wg_bias.grad 
#dhc

In [40]:
dwc = torch.transpose(torch.mm(torch.transpose(iph,0,1),dhc),0,1)
#model.dec_lstm.wg_weights.grad 
#dwc

In [41]:
dhi = torch.tanh(hg) * dc
dhi = nn.Sigmoid()(hi) * (1 - nn.Sigmoid()(hi)) * dhi
#model.dec_lstm.wi_bias.grad 
#dhi

In [42]:
dwi = torch.transpose(torch.mm(torch.transpose(iph,0,1),dhi),0,1)
#model.dec_lstm.wi_weights.grad 
#dwi

In [43]:
# GOOD TO HERE

In [44]:
wf = model.dec_lstm.wf_weights
wi = model.dec_lstm.wi_weights
wg = model.dec_lstm.wg_weights
wo = model.dec_lstm.wo_weights

In [45]:
dXf = torch.mm(dhf, wf)
dXi = torch.mm(dhi, wi)
dXg = torch.mm(dhc, wg)
dXo = torch.mm(dho, wo)

dX = dXf + dXi + dXg + dXo

In [55]:
dXg # g

tensor([[ 1.6595e-05, -2.7810e-06, -1.6969e-05,  8.0342e-06,  2.2093e-05,
          1.9435e-05, -1.6745e-05, -1.7095e-05,  7.1709e-06,  4.6269e-07,
          2.9443e-06,  6.2831e-06, -1.3735e-07,  2.1463e-05, -2.5463e-05,
          3.6913e-05, -3.6688e-07, -4.2045e-06, -3.2695e-05, -9.4703e-06,
         -2.6375e-05,  2.2605e-05,  1.4875e-05,  9.8012e-06,  1.6359e-05,
         -1.5928e-05,  5.0553e-06, -1.1262e-04, -9.2154e-06, -5.3050e-06,
          2.9069e-05,  3.7993e-05,  2.8332e-05,  1.3348e-05,  2.2708e-05,
         -3.7251e-05, -2.5965e-05,  2.5318e-05, -4.7135e-06, -1.0470e-06,
          2.4440e-05, -2.2680e-06,  1.3107e-06, -2.0752e-05, -2.9002e-05,
         -3.5041e-05, -4.5897e-06, -3.4172e-06,  5.9837e-06, -1.1329e-05,
         -2.4003e-06, -9.1842e-06, -1.2023e-05, -1.0342e-05, -1.2649e-05,
         -1.0187e-04,  6.1012e-05,  1.2240e-05, -1.4225e-05, -8.3145e-06,
         -1.5363e-06,  4.4611e-05, -6.2503e-05, -1.8145e-06,  2.8968e-05,
          6.6005e-05, -2.4075e-05,  2.

In [47]:
dX

tensor([[ 1.4193e-05, -2.8273e-08, -1.5461e-05,  2.4739e-06,  7.5731e-06,
          1.5298e-05, -1.3739e-05, -1.0707e-05,  1.0929e-05, -1.2874e-05,
          1.6749e-06, -1.6916e-05, -6.0334e-06,  2.2122e-05, -8.8243e-06,
          1.1446e-05, -4.7403e-05,  4.9437e-06, -4.0524e-05, -2.6474e-05,
         -5.0214e-05,  6.2971e-05,  3.9856e-05,  9.1130e-06,  2.1419e-05,
         -9.0942e-05, -2.7364e-06, -1.9318e-04, -2.3158e-05, -1.0194e-05,
          7.3917e-05,  2.6020e-05, -5.1513e-06,  2.1701e-05,  2.0251e-05,
         -6.7672e-05, -3.9842e-05,  4.3978e-05, -3.2675e-05,  8.5161e-06,
          2.7519e-05, -1.7729e-05,  1.4941e-05, -1.2798e-05, -4.7800e-05,
         -5.0332e-05, -9.7576e-06, -1.6488e-05,  1.1328e-05, -1.5349e-05,
          6.9982e-06, -3.3907e-06, -2.8387e-06,  9.0619e-06, -1.9349e-05,
         -1.1723e-04,  5.6724e-05,  2.7596e-05, -1.2638e-05,  9.4847e-06,
         -2.1525e-05,  9.9215e-05, -4.8984e-05,  1.3287e-05,  2.0034e-05,
          1.1207e-04, -4.8469e-05, -1.

In [48]:
dX.shape

torch.Size([1, 266])

In [ ]:
dinp, dhid = torch.split(dX, [10,256], dim=1)
dinp.shape, dhid.shape

In [ ]:
def lstm_backward(prob, y_train, d_next, cache):
    # Unpack the cache variable to get the intermediate variables used in forward step
    ... = cache
    dh_next, dc_next = d_next

    # Softmax loss gradient
    dy = prob.copy() # CHECK
    dy[1, y_train] -= 1. # CHECK

    # Hidden to output gradient
    dWy = h.T @ dy # CHECK
    dby = dy # CHECK
    # Note we're adding dh_next here
    dh = dy @ Wy.T + dh_next # CHECK

    # Gradient for ho in h = ho * tanh(c)
    dho = tanh(c) * dh # CHECK
    dho = dsigmoid(ho) * dho # CHECK

    # Gradient for c in h = ho * tanh(c), note we're adding dc_next here
    dc = ho * dh * dtanh(c) # CHECK
    dc = dc + dc_next # CHECK

    # Gradient for hf in c = hf * c_old + hi * hc
    dhf = c_old * dc # CHECK
    dhf = dsigmoid(hf) * dhf # CHECK

    # Gradient for hi in c = hf * c_old + hi * hc
    dhi = hc * dc # CHECK
    dhi = dsigmoid(hi) * dhi # CHECK

    # Gradient for hc in c = hf * c_old + hi * hc
    dhc = hi * dc # CHECK
    dhc = dtanh(hc) * dhc # CHECK

    # Gate gradients, just a normal fully connected layer gradient
    dWf = X.T @ dhf # CHECK
    dbf = dhf # CHECK
    dXf = dhf @ Wf.T # CHECK

    dWi = X.T @ dhi # CHECK
    dbi = dhi # CHECK
    dXi = dhi @ Wi.T # CHECK

    dWo = X.T @ dho # CHECK
    dbo = dho # CHECK
    dXo = dho @ Wo.T # CHECK

    dWc = X.T @ dhc # CHECK
    dbc = dhc # CHECK
    dXc = dhc @ Wc.T # CHECK

    # As X was used in multiple gates, the gradient must be accumulated here
    dX = dXo + dXc + dXi + dXf # CHECK
    # Split the concatenated X, so that we get our gradient of h_old
    dh_next = dX[:, :H]
    # Gradient for c_old in c = hf * c_old + hi * hc
    dc_next = hf * dc

    grad = dict(Wf=dWf, Wi=dWi, Wc=dWc, Wo=dWo, Wy=dWy, bf=dbf, bi=dbi, bc=dbc, bo=dbo, by=dby)
    state = (dh_next, dc_next)

    return grad, state